# Aula 4: WebSockets em Python

Bem-vindo à quarta aula do curso de APIs em Python. Hoje abordaremos comunicação em tempo real usando WebSockets:

- O que são WebSockets e como diferem do HTTP
- Protocolo de handshake e comunicação full-duplex
- Biblioteca `websockets` em Python: servidor e cliente
- Integração com Flask usando Flask-SocketIO
- Exemplo prático: chat em tempo real
- Gerenciamento de salas e broadcast
- Tratamento de eventos e reconexão
- Testando a aplicação

## 1. Introdução aos WebSockets

- WebSockets oferecem comunicação full-duplex sobre uma única conexão TCP.
- Diferem do HTTP: após handshake, mantêm conexão aberta, permitindo envio de mensagens em tempo real entre cliente e servidor.
- Ideal para chat, notificações em tempo real, dashboards ao vivo.

### Fluxo do Handshake

1. Cliente envia requisição HTTP com `Upgrade: websocket`.
2. Servidor responde com código `101 Switching Protocols`.
3. A conexão é atualizada para protocolo WebSocket, iniciando comunicação bidirecional.

## 2. Usando a biblioteca `websockets`

Instalação:

```bash
pip install websockets
```

### 2.1. Servidor WebSocket simples

In [ ]:
import asyncio
import websockets

async def echo(websocket, path):
    async for message in websocket:
        print(f"Recebido: {message}")
        await websocket.send(f"Echo: {message}")

start_server = websockets.serve(echo, 'localhost', 8765)

asyncio.get_event_loop().run_until_complete(start_server)
print("Servidor WebSocket rodando em ws://localhost:8765")
asyncio.get_event_loop().run_forever()

**Explicação**

- `websockets.serve` cria servidor escutando na porta 8765.
- Handler `echo` recebe mensagens e envia de volta.
- `async for message`: mantém conexão lendo mensagens até desconexão.

### 2.2. Cliente WebSocket simples

In [ ]:
import asyncio
import websockets

async def hello():
    uri = "ws://localhost:8765"
    async with websockets.connect(uri) as websocket:
        await websocket.send("Olá servidor!")
        response = await websocket.recv()
        print(f"Resposta: {response}")

asyncio.get_event_loop().run_until_complete(hello())

## 3. Integração com Flask usando Flask-SocketIO

Instalação:

```bash
pip install flask-socketio
pip install eventlet  # backend assíncrono para produção
```

In [ ]:
# app.py
from flask import Flask, render_template
from flask_socketio import SocketIO, send, emit, join_room, leave_room

app = Flask(__name__)
app.config['SECRET_KEY'] = 'chave_secreta'
socketio = SocketIO(app, async_mode='eventlet')

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('message')
def handle_message(msg):
    print(f"Mensagem recebida: {msg}")
    send(msg, broadcast=True)

@socketio.on('join')
def on_join(data):
    room = data['room']
    join_room(room)
    send(f"Entrou na sala {room}", room=room)

@socketio.on('leave')
def on_leave(data):
    room = data['room']
    leave_room(room)
    send(f"Saiu da sala {room}", room=room)

if __name__ == '__main__':
    socketio.run(app, host='0.0.0.0', port=5000)

## 4. Exemplo prático de chat em tempo real

Crie o arquivo `templates/index.html` com o cliente Socket.IO:

```html
<!DOCTYPE html>
<html>
<head>
  <title>Chat em Tempo Real</title>
</head>
<body>
  <ul id="messages"></ul>
  <input id="input" autocomplete="off"/><button id="send">Enviar</button>
  <script src="https://cdn.socket.io/4.5.0/socket.io.min.js"></script>
  <script>
    const socket = io();
    const input = document.getElementById('input');
    document.getElementById('send').onclick = () => {
      socket.send(input.value);
      input.value = '';
    };
    socket.on('message', (msg) => {
      const li = document.createElement('li');
      li.textContent = msg;
      document.getElementById('messages').append(li);
    });
  </script>
</body>
</html>
```

## 5. Gerenciamento de Salas e Broadcast

- Use `join_room(room)` e `leave_room(room)` para gerenciar salas.
- `send(..., room=room)` para enviar a um room específico.
- `broadcast=True` envia para todos conectados.

## 6. Tratamento de Eventos e Reconexão

- Defina eventos customizados: `@socketio.on('my_event')`.
- Reconexão automática no cliente Socket.IO.
- Adicione timeouts e heartbeats configuráveis.

## 7. Testando a Aplicação

1. Inicie o servidor: `python app.py`.
2. Abra `http://localhost:5000/` em várias abas para simular múltiplos clientes.
3. Envie mensagens e veja atualização em tempo real.

---

Parabéns! Você agora domina WebSockets em Python e pode criar aplicações real-time.